<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

In this exercise we are using high level abstractions from torch.nn like nn.Linear.
Note: during the next lab session we will go one level deeper and implement more things
with bare hands.

Tasks:

    1. Read the code.

    2. Check that the given implementation reaches 95% test accuracy for architecture input-128-128-10 after few epochs.

    3. Add the option to use SGD with momentum instead of ADAM.

    4. Experiment with different learning rates, plot the learning curves for different
    learning rates for both ADAM and SGD with momentum.

    5. Parameterize the constructor by a list of sizes of hidden layers of the MLP.
    Note that this requires creating a list of layers as an atribute of the Net class,
    and one can't use a standard python list containing nn.Modules (why?).
    Check torch.nn.ModuleList.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [18]:
class Net(nn.Module):
    def __init__(self, layers):
        super(Net, self).__init__()
        # After flattening an image of size 28x28 we have 784 inputs
        self.layers = nn.ModuleList([nn.Linear(x, y) for x, y in zip(layers, layers[1:])])

    def forward(self, x):
        a = torch.flatten(x, 1)
        x = a
        for l in self.layers:
          x = l(a)
          a = F.relu(x)

        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [3]:
batch_size = 256
test_batch_size = 1000
epochs = 5
lr = 1e-2
seed = 1
log_interval = 10
use_cuda = torch.cuda.is_available()

In [4]:
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [5]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

100%|██████████| 9912422/9912422 [00:00<00:00, 107867824.27it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 32105935.28it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 28010673.86it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5542778.23it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
model = Net([784, 128, 128, 10]).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299619
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.529521
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.497725
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.426544
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.253610
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.349574
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.239824
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.174905
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.216088
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.247328
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.273255
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.237411
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.173276
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.196686
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.256425
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.207012
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.162990
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.147656
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.254312
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.076072
Train Epoc

In [8]:
model = Net([784, 128, 128, 10]).to(device)
SGD_optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, SGD_optimizer, epoch, log_interval)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307556
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.198563
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.883218
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.368348
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.735437
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.693011
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.435643
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.365278
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.459646
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.378412
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.356934
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.497564
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.409239
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.394924
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.326487
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.350912
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.306210
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.257856
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.424362
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.187470
Train Epoc